In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
currdir = os.getcwd()
regex_name = r'(([A-Z])\w+)'
batchrun = '600s-run1'

In [3]:
SURVEY_ZONE_MAPPING = {'30': (2, 2, 0), '22': (3, 1, 2), '2c': (2, 0, 0), '28': (2, 2, 1), '25': (3, 2, 1), '26': (3, 1, 1), '27': (3, 0, 1), '20': (2, 2, 2), '21': (3, 2, 2), '1e': (3, 1, 2), '23': (3, 0, 2), '24': (2, 0, 1), '29': (3, 2, 1), '1a': (3, 1, 3), '0': (1, 0, 3), '3': (3, 1, 0), '2': (3, 2, 0), '5': (1, 0, 0), '1d': (3, 2, 2), '7': (2, 1, 1), '1f': (3, 0, 2), '9': (3, 1, 1), '8': (3, 2, 1), '3a': (3, 1, 3), '1c': (2, 0, 2), '4': (3, 0, 0), 'a': (3, 0, 1), '6': (1, 0, 1), '39': (3, 2, 3), '12': (1, 0, 2), '3b': (3, 0, 3), '1b': (3, 0, 3), 'b': (1, 0, 2), '13': (1, 0, 2), 'd': (3, 2, 2), '11': (1, 0, 3), '10': (1, 0, 3), 'c': (2, 1, 2), '38': (2, 0, 3), '15': (1, 0, 1), '14': (1, 0, 1), '17': (1, 0, 0), 'f': (3, 0, 2), '19': (3, 2, 3), '32': (3, 1, 0), '31': (3, 2, 0), '16': (1, 0, 0), '37': (3, 0, 3), '36': (3, 1, 3), '35': (3, 2, 3), '34': (2, 1, 3), '2d': (3, 2, 0), '2e': (3, 1, 0), '2f': (3, 0, 0), '1': (2, 1, 0), '2a': (3, 1, 1), '2b': (3, 0, 1), '18': (2, 2, 3), '33': (3, 0, 0), 'e': (3, 1, 2)}

## Checking out the Excel sheet (Obtaining list of columns)

In [4]:
filename = currdir + '/experiments/sim-results/result_d450_a0.5_b0.5_c0.xls'
df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

In [5]:
df_tmp.head()

,t,0x37 - approach3_lane2,0x7 - approach1_lane2,0x25 - approach1_lane1,0x19 - approach3_lane3,0x32 - approach0_lane3,0x31 - approach0_lane3,0x28 - approach1_lane3,0x17 - exit0_lane1,0x1d - approach2_lane1,...,0x15 - exit1_lane1,0x10 - exit3_lane3,0x12 - exit2_lane3,0x13 - exit2_lane1,0x14 - exit1_lane3,0x2e - approach0_lane1,0x18 - approach3_lane3,0xc - approach2_lane2,0xb - exit2_lane2,0x2a - approach1_lane3
0,60,48.302682,13.273043,26.976835,43.539097,55.568291,44.944941,20.243368,103.680000,37.538341,...,51.840000,54.432000,45.360000,82.080000,53.71200,49.717895,19.150222,11.186308,67.885714,49.100936
1,120,47.894655,12.090297,19.823233,48.204000,51.956352,43.296960,9.790429,55.827692,34.585888,...,51.840000,50.438919,54.432000,60.192000,53.71200,50.010353,19.150222,5.427882,61.714286,47.096816
2,180,39.172562,9.701695,20.636381,47.028293,53.933929,44.602721,9.276646,53.760000,24.142306,...,54.093913,53.460000,52.338462,60.800000,55.95000,48.755613,17.158938,5.759287,62.400000,42.567083
3,240,41.321466,8.300053,16.285228,47.707052,54.024727,44.756887,10.772007,53.169231,12.507938,...,55.827692,54.154286,54.432000,56.824615,57.17729,49.285565,22.069463,5.858728,60.252632,46.204831
4,300,43.525725,7.429128,13.314747,49.229617,55.012608,45.240632,11.140833,55.013878,6.633931,...,54.889412,54.432000,56.700000,55.246154,57.29280,43.671799,22.069463,5.155285,59.678351,46.527097


In [15]:
# Filter out some of the rows

delay_rows = ["{} Delay".format(d) for d in range(60,601,60)]
delay_rename = dict([
    ("{} Delay".format(d), d)
    for d in range(60,601,60)
])

thru_rows = ["{} Throughput".format(d) for d in range(60,601,60)]
thru_rename = dict([
    ("{} Throughput".format(d), d)
    for d in range(60,601,60)
])

vol_rows = [d for d in range(60,601,60)]

In [7]:
# Filter out some of the columns

survey_zones = df_tmp.columns.values[1:]

entrances = [x for x in survey_zones if 'approach' in x]
exits = [y for y in survey_zones if 'exit' in y]

print("{} approach zones, {} exit zones".format(len(entrances), len(exits)))

48 approach zones, 12 exit zones


In [8]:
reg_approach = re.compile('0x(\w+) - approach.*')
reg_exit = re.compile('0x(\w+) - exit.*')

entrances_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_approach.match(k).group(1)])
    for k in entrances
])

exits_rename = dict([
    (k, SURVEY_ZONE_MAPPING[reg_exit.match(k).group(1)])
    for k in exits
])

## Checking out the Volume sheet

In [9]:
df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)

In [10]:
df_tmp

,t,0x1c - approach2_lane1,0x1d - approach2_lane1,0x1e - approach2_lane1,0x1f - approach2_lane1,0x28 - approach1_lane3,0x29 - approach1_lane3,0x2a - approach1_lane3,0x2b - approach1_lane3,0x14 - exit1_lane3,...,0x13 - exit2_lane1,0x30 - approach0_lane3,0x31 - approach0_lane3,0x32 - approach0_lane3,0x33 - approach0_lane3,0x7 - approach1_lane2,0x8 - approach1_lane2,0x9 - approach1_lane2,0xa - approach1_lane2,0x5 - exit0_lane2
0,60,4.000000,4.000000,4.000,4.000,5.000000,6.000000,6.000000,6.000000,8.000000,...,4.000000,8.000000,8.000000,9.000000,9.000,14.000,15.000000,15.000,15.00,6.000000
1,120,4.000000,5.000000,6.000,6.000,11.000000,12.000000,12.000000,13.000000,6.000000,...,8.000000,9.000000,9.000000,8.000000,8.000,21.000,21.000000,23.000,24.00,14.000000
2,180,8.000000,8.000000,7.000,7.000,12.000000,10.000000,10.000000,9.000000,11.000000,...,8.000000,6.000000,6.000000,7.000000,7.000,19.000,20.000000,19.000,19.00,11.000000
3,240,5.000000,8.000000,9.000,9.000,9.000000,9.000000,10.000000,10.000000,8.000000,...,8.000000,4.000000,5.000000,4.000000,5.000,22.000,23.000000,22.000,22.00,12.000000
4,300,5.000000,5.000000,4.000,4.000,8.000000,8.000000,7.000000,7.000000,7.000000,...,8.000000,9.000000,8.000000,8.000000,7.000,12.000,9.000000,9.000,8.00,14.000000
5,360,5.000000,3.000000,4.000,5.000,3.000000,4.000000,4.000000,5.000000,5.000000,...,8.000000,7.000000,8.000000,8.000000,8.000,15.000,16.000000,16.000,16.00,14.000000
6,420,7.000000,7.000000,7.000,7.000,5.000000,4.000000,4.000000,3.000000,4.000000,...,10.000000,3.000000,2.000000,2.000000,2.000,9.000,9.000000,10.000,11.00,12.000000
7,480,8.000000,7.000000,6.000,5.000,4.000000,4.000000,4.000000,4.000000,4.000000,...,10.000000,4.000000,4.000000,5.000000,6.000,14.000,15.000000,15.000,14.00,12.000000
8,540,6.000000,5.000000,6.000,7.000,5.000000,6.000000,7.000000,7.000000,6.000000,...,9.000000,6.000000,6.000000,5.000000,4.000,14.000,14.000000,15.000,16.00,12.000000
9,600,9.000000,10.000000,10.000,9.000,5.000000,4.000000,3.000000,3.000000,3.000000,...,10.000000,5.000000,5.000000,6.000000,7.000,13.000,15.000000,15.000,17.00,14.000000


In [22]:
vol_rows = [d for d in range(60,601,60)]

df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
df_volume

## ADD THIS LATER WHEN RESULTS ARE OUT

level_1,"(2, 0, 0)","(2, 0, 1)","(2, 0, 2)","(2, 0, 3)","(2, 1, 0)","(2, 1, 1)","(2, 1, 2)","(2, 1, 3)","(2, 2, 0)","(2, 2, 1)",...,"(3, 0, 2)","(3, 0, 3)","(3, 1, 0)","(3, 1, 1)","(3, 1, 2)","(3, 1, 3)","(3, 2, 0)","(3, 2, 1)","(3, 2, 2)","(3, 2, 3)"
t,,,,,,,,,,,,,,,,,,,,,
60,2.361421,4.995117,3.997856,25.785600,11.284286,11.168780,17.312143,7.386514,0.000000,5.494629,...,164.253363,162.749627,164.425067,150.327829,144.293539,155.266953,135.035027,113.481455,91.944277,103.480093
120,2.961000,4.100469,3.006140,7.367314,11.570366,5.987448,8.363278,7.159237,22.387765,10.068691,...,139.333132,123.743693,151.077503,132.955402,124.987208,132.121189,106.925841,82.691021,70.839872,80.060153
180,2.852067,3.901983,2.935347,8.362897,10.110720,6.233523,7.186657,4.964620,20.148988,9.404010,...,139.085440,140.445710,138.666017,129.636961,127.179979,132.328272,96.840244,79.591799,75.107046,79.963017
240,2.857949,4.380683,3.167804,6.131941,9.158261,5.880128,7.790464,4.148987,20.480287,9.526291,...,147.665284,137.367651,128.830114,130.058382,133.319181,125.839781,89.184434,80.080547,80.833101,73.774393
300,2.812950,4.817064,3.192714,5.966722,8.888359,6.113752,7.947298,4.328036,18.123429,9.526291,...,151.253349,131.913759,113.678785,132.436363,134.263882,119.974622,85.883132,83.558537,82.562028,64.545200
360,2.700449,4.701286,3.016532,5.512361,8.578100,5.289249,7.806754,4.315650,17.873105,10.342830,...,154.863171,129.737922,106.044138,130.786554,135.582174,114.287153,79.295800,80.132618,84.219947,59.729574
420,2.755511,5.041888,4.033037,4.181449,7.982147,5.667327,8.659206,4.169806,14.382837,9.615600,...,141.126796,134.463422,101.933945,118.398455,126.855039,107.470945,76.790851,76.125413,88.294345,46.978246
480,3.034978,4.671547,4.694187,3.755184,6.426305,4.733023,7.490782,4.491799,13.455273,10.869809,...,130.493377,129.956725,88.385699,101.319184,112.987324,108.193165,58.463174,62.411878,73.944373,50.687811
540,3.120949,3.373895,4.812572,3.562484,5.142320,4.116244,5.811383,5.707761,11.483379,12.928538,...,111.150186,126.016481,75.579808,81.703603,88.039304,105.637200,50.137537,59.756763,54.701770,53.878180


## Preprocessing the results

In [18]:
weightlist = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (0.5, 0.5, 0),
    (0, 0.5, 0.5),
    (0.5, 0, 0.5),
    (0.33, 0.33, 0.33),
    (0, 0, 0)
]

demands = [
    450,
    900,
    (450, 900),
    (900, 1800)
]

_model_type = {
    (1, 0, 0): 'Delay priority',
    (0, 1, 0): 'Throughput priority',
    (0, 0, 1): 'Flow priority',
    (0.5, 0.5, 0): 'Delay-Throughput priority',
    (0, 0.5, 0.5): 'Throughput-Flow priority',
    (0.5, 0, 0.5): 'Delay-Flow priority',
    (0.33, 0.33, 0.33): 'Equal priority',
    (0, 0, 0): 'Parent model',
}

In [24]:
total_thru_dfs = []
total_delay_dfs = []
ave_delay_dfs = []

for demand in demands:
    for weights in weightlist:

        partial_fn = 'old' if weights == (0, 0, 0) else 'a{}_b{}_c{}'.format(*weights)
        filename = currdir + '/experiments/sim-results/result_d{}_{}.xls'.format(demand, partial_fn)
        df_tmp = pd.read_excel(filename, sheet_name='Speed').fillna(0)

        # Get the cumulative delays

        df_tmp1 = df_tmp[df_tmp.t.isin(delay_rows)]
        df_tmp1['t'].replace(delay_rename, inplace=True)
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_cumdelay = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_cumdelay.to_pickle(currdir + '/experiments/cumu-delay/delay_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        
        # Get the cumulative throughputs

        df_tmp3 = df_tmp[df_tmp.t.isin(thru_rows)]
        if weights == (0, 0, 0):
            for col in ['0x11 - exit3_lane1', '0x15 - exit1_lane1', '0x17 - exit0_lane1', '0x13 - exit2_lane1']:
                if col not in df_tmp3.columns:
                    df_tmp3[col] = 0
        df_tmp3['t'].replace(thru_rename, inplace=True)
        df_tmp3 = df_tmp3.sort_values(by='t').set_index('t')[exits].stack().reset_index()
        df_tmp3['level_1'] = df_tmp3['level_1'].map(exits_rename)
        df_tmp3 = df_tmp3.groupby(['t', 'level_1']).sum().reset_index()

        df_cumthru = df_tmp3.pivot(index='t', columns='level_1', values=0)
        df_cumthru.to_pickle(currdir + '/experiments/cumu-thru/thru_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))
        
        # Get volumes
        
        df_tmp = pd.read_excel(filename, sheet_name='Volume').fillna(0)
        df_tmp1 = df_tmp[df_tmp.t.isin(vol_rows)]
        df_tmp1 = df_tmp1.sort_values(by='t').set_index('t')[entrances].stack().reset_index()
        df_tmp1['level_1'] = df_tmp1['level_1'].map(entrances_rename)
        df_tmp1 = df_tmp1.groupby(['t', 'level_1']).sum().reset_index()

        df_volume = df_tmp1.pivot(index='t', columns='level_1', values=0)
        df_volume.to_pickle(currdir + '/experiments/actual-volume/vol_d{}_a{}_b{}_c{}.pkl'.format(demand, *weights))

        # Get aggregate values

        df_total_thru_tmp = df_cumthru.loc[600, :].reset_index().set_index('level_1').T
        df_total_thru_tmp['demand'] = [demand]
        df_total_thru_tmp['new_model'] = weights != (0, 0, 0)
        df_total_thru_tmp['alpha'] = weights[0]
        df_total_thru_tmp['beta'] = weights[1]
        df_total_thru_tmp['gamma'] = weights[2]
        df_total_thru_tmp['model_type'] = _model_type[weights]
        total_thru_dfs.append(df_total_thru_tmp)

        df_total_delay_tmp = df_cumdelay.loc[600, :].reset_index().set_index('level_1').T
        df_total_delay_tmp['demand'] = [demand]
        df_total_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_total_delay_tmp['alpha'] = weights[0]
        df_total_delay_tmp['beta'] = weights[1]
        df_total_delay_tmp['gamma'] = weights[2]
        df_total_delay_tmp['model_type'] = _model_type[weights]
        total_delay_dfs.append(df_total_delay_tmp)

        df_tmp2 = df_tmp[df_tmp.t == 'Average Delay'][entrances].rename(columns=entrances_rename).stack().reset_index()
        df_ave_delay_tmp = df_tmp2.groupby('level_1').mean().reset_index().pivot(index='level_0', columns='level_1', values=0)
        df_ave_delay_tmp['demand'] = [demand]
        df_ave_delay_tmp['new_model'] = weights != (0, 0, 0)
        df_ave_delay_tmp['alpha'] = weights[0]
        df_ave_delay_tmp['beta'] = weights[1]
        df_ave_delay_tmp['gamma'] = weights[2]
        df_ave_delay_tmp['model_type'] = _model_type[weights]
        ave_delay_dfs.append(df_ave_delay_tmp)

/home/franciszacdlc/.virtualenvs/localsim2.7/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
df_total_thru = pd.concat(total_thru_dfs).reset_index().iloc[:, 1:]
df_total_thru.to_pickle(currdir + '/experiments/total_throughput.pkl')

df_total_delay = pd.concat(total_delay_dfs).reset_index().iloc[:, 1:]
df_total_delay.to_pickle(currdir + '/experiments/total_delay.pkl')

df_ave_delay = pd.concat(ave_delay_dfs).reset_index().iloc[:, 1:]
df_ave_delay.to_pickle(currdir + '/experiments/ave_delay.pkl')